# Opening a Chinese restaurant in Edinburgh

# Import libraries

In [17]:
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
import numpy as np
print("Libraries imported and installed")

Libraries imported and installed


# read data and pre-processing

## Plot popular area in Edinburgh

In [19]:
!wget --quiet https://martinjc.github.io/UK-GeoJSON/json/sco/wards_by_lad/topo_S12000036.json -O topo_S12000036.json  

edi_geo = r'topo_S12000036.json'

# create a plain world map
edi_map = folium.Map(location=[55.95, -3.20], zoom_start=12)

edi_map.choropleth(
    geo_data=edi_geo,
#    data=df_district,
    columns=['Neighborhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name=''
)

# display map
edi_map

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [2]:
data = [['Old_Town', 55.948726, -3.186815], 
        ['New_Town', 55.952972, -3.200384], 
        ['Leith', 55.975485, -3.170548], 
        ['Stockbridge', 55.958805, -3.210040], 
        ['Bruntsfield', 55.935706, -3.209900],
        ['Haymaket', 55.946128, -3.216803],
        ['Southside', 55.939108, -3.179524]]
  
df = pd.DataFrame(data, columns = ['Region', 'Latitude', 'Longitude']) 

print (df)

        Region   Latitude  Longitude
0     Old_Town  55.948726  -3.186815
1     New_Town  55.952972  -3.200384
2        Leith  55.975485  -3.170548
3  Stockbridge  55.958805  -3.210040
4  Bruntsfield  55.935706  -3.209900
5     Haymaket  55.946128  -3.216803
6    Southside  55.939108  -3.179524


In [3]:
map_edinburgh = folium.Map(location=[55.951947, -3.193543], zoom_start=12)

for lat, lng, region in zip(df['Latitude'], df['Longitude'], df['Region']):
    label = region
    label = folium.Popup(region, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edinburgh)  
    
map_edinburgh

## Using foursquare to get nearby venues

In [4]:
CLIENT_ID = '3N0SR5V4HMX20CMRYQEB2KWYEJTQAYBOL20DCVHP3TEMCF13' # your Foursquare ID
CLIENT_SECRET = 'LAR3Y5LPBH4SWJ5PBQY2K1E1MI2QZKQXZ2SRUEGNLSOVB3JU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3N0SR5V4HMX20CMRYQEB2KWYEJTQAYBOL20DCVHP3TEMCF13
CLIENT_SECRET:LAR3Y5LPBH4SWJ5PBQY2K1E1MI2QZKQXZ2SRUEGNLSOVB3JU


In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## All venues in toronto

In [6]:
edi_venues = getNearbyVenues(names=df['Region'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Old_Town
New_Town
Leith
Stockbridge
Bruntsfield
Haymaket
Southside


In [7]:
print(edi_venues.shape)
edi_venues.head()

(369, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Old_Town,55.948726,-3.186815,Salt Horse,55.949180,-3.185647,Beer Bar
1,Old_Town,55.948726,-3.186815,The Royal Mile,55.950029,-3.188567,Road
2,Old_Town,55.948726,-3.186815,BrewDog Edinburgh,55.948507,-3.189626,Beer Bar
3,Old_Town,55.948726,-3.186815,Brew Lab Coffee,55.947132,-3.186685,Coffee Shop
4,Old_Town,55.948726,-3.186815,National Museum of Scotland,55.947254,-3.189315,Museum


In [8]:
edi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bruntsfield,31,31,31,31,31,31
Haymaket,43,43,43,43,43,43
Leith,43,43,43,43,43,43
New_Town,72,72,72,72,72,72
Old_Town,100,100,100,100,100,100
Southside,43,43,43,43,43,43
Stockbridge,37,37,37,37,37,37


In [9]:
print('There are {} uniques categories.'.format(len(edi_venues['Venue Category'].unique())))

There are 107 uniques categories.


## one hot encoding

In [10]:
# one hot encoding
edi_onehot = pd.get_dummies(edi_venues[['Venue Category']], prefix="", prefix_sep="")
edi_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
edi_onehot.insert(loc=0, column='Neighborhood', value=edi_venues['Neighborhood'] )
edi_onehot.shape

(369, 107)

In [11]:
edi_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,...,Thai Restaurant,Theater,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Well,Whisky Bar,Wine Bar,Wings Joint
0,Old_Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Old_Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Old_Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Old_Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Old_Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
edi_grouped = edi_onehot.groupby('Neighborhood').mean().reset_index()
edi_grouped.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,...,Thai Restaurant,Theater,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Well,Whisky Bar,Wine Bar,Wings Joint
0,Bruntsfield,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.064516,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
1,Haymaket,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.069767,...,0.000000,0.00,0.000000,0.023256,0.000000,0.023256,0.0,0.000000,0.000000,0.00
2,Leith,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.023256,0.116279,...,0.000000,0.00,0.023256,0.000000,0.023256,0.000000,0.0,0.023256,0.023256,0.00
3,New_Town,0.00,0.013889,0.0,0.00,0.027778,0.013889,0.000000,0.000000,0.097222,...,0.013889,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.013889,0.000000,0.00
4,Old_Town,0.01,0.000000,0.0,0.03,0.000000,0.000000,0.010000,0.000000,0.050000,...,0.010000,0.01,0.000000,0.000000,0.010000,0.000000,0.0,0.020000,0.010000,0.01


In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = edi_grouped['Neighborhood']

for ind in np.arange(edi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(edi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bruntsfield,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Grocery Store,Restaurant,Bakery,Fish & Chips Shop,Chinese Restaurant,Breakfast Spot
1,Haymaket,Hotel,Bar,Italian Restaurant,Pub,Restaurant,Café,Grocery Store,Coffee Shop,Pizza Place,Sandwich Place
2,Leith,Bar,Café,Pub,Seafood Restaurant,Hotel,Deli / Bodega,Coffee Shop,French Restaurant,Trail,Discount Store
3,New_Town,Bar,Café,Cocktail Bar,Seafood Restaurant,Coffee Shop,Hotel,Park,Mexican Restaurant,Speakeasy,Italian Restaurant
4,Old_Town,Pub,Hotel,Café,Bar,Restaurant,Coffee Shop,Museum,Pizza Place,Art Gallery,Scottish Restaurant


## Cluster neighborhoods

In [15]:
# set number of clusters
kclusters = 4

edi_grouped_clustering = edi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(edi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([2, 1, 1, 1, 1, 0, 3], dtype=int32)

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

edi_merged = edi_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
edi_merged = edi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

edi_merged.head() # check the last columns!

NameError: name 'edi_data' is not defined

## Map of neighborhoods in different clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(edi_merged['Latitude'], edi_merged['Longitude'], edi_merged['Neighborhood'], edi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [ ]:
edi_merged.loc[edi_merged['Cluster Labels'] == 0, edi_merged.columns[[1] + list(range(5, edi_merged.shape[1]))]]

## Cluster 2

In [ ]:
edi_merged.loc[edi_merged['Cluster Labels'] == 1, edi_merged.columns[[1] + list(range(5, edi_merged.shape[1]))]]

## Cluster 3

In [ ]:
edi_merged.loc[edi_merged['Cluster Labels'] == 2, edi_merged.columns[[1] + list(range(5, edi_merged.shape[1]))]]

## Cluster 4

In [ ]:
edi_merged.loc[toronto_merged['Cluster Labels'] == 3, edi_merged.columns[[1] + list(range(5, edi_merged.shape[1]))]]

# Observation

### Cluster 3 is the largest cluster in toronto. There are a lot of shops, cafe nearby the neighborhoods in this cluster. 


### Cluster 2 neighbors have parks surrounding them.


# The end